In [25]:
import pandas as pd
import numpy as np
import scipy as scp
import random as rd
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn import datasets
from sklearn.covariance import LedoitWolf
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

## Data of OAD Cleaned

In [9]:
#Initialize dataframe from excel spreadsheet
oad = pd.read_excel("oxy_avail_dataset.xls")
oad.head()

,OTUId,45A 10,45B 10,45C 10,75A 10,75B 10,75C 10,150A 10,150B 10,150C 10,...,class_confidence,order_,order_confidence,family,family_confidence,genus,genus_confidence,species,species_confidence,sequence
0,OTU_1,24014,12430,477,231,35910,9801,6183,22212,5628,...,1.0,Methylophilales,1.0,Methylophilaceae,1.0,Methylophilus,1.00,NaN,NaN,ATTGAACGCTGGCGGAATGCTTTACACATGCAAGTCGAACGATGAA...
1,OTU_2,20188,24781,13429,22430,22253,24440,684,688,622,...,1.0,Methylococcales,1.0,Methylococcaceae,1.0,Methylobacter,1.00,NaN,NaN,ATTGAACGCTGGCGGTATGCTTAACACATGCAAGTCGAACGGTAGC...
2,OTU_3,2417,5259,29605,790,591,8685,1698,318,105,...,1.0,Methylophilales,1.0,Methylophilaceae,1.0,Methylotenera,1.00,NaN,NaN,ATTGAACGCTGGCGGAATGCTTTACACATGCAAGTCGAACGATGAT...
3,OTU_5,289,363,208,259,366,331,18787,10820,5755,...,1.0,Methylococcales,1.0,Methylococcaceae,1.0,Methylosarcina,1.00,NaN,NaN,ATTGAACGCTGGCGGTATGCTTAACACATGCAAGTCGAACGGTAAC...
4,OTU_4,98,98,5834,856,361,192,72,199,72,...,1.0,Methylophilales,1.0,Methylophilaceae,1.0,Methylotenera,0.95,NaN,NaN,ATTGAACGCTGGCGGAATGCTTTACACATGCAAGTCGAACGGCAGC...


In [10]:
#Drop rows that describe the index and set the index
oad_cleaned = oad.loc[:, :'225C 16'].set_index('OTUId')
oad_cleaned.head()

,45A 10,45B 10,45C 10,75A 10,75B 10,75C 10,150A 10,150B 10,150C 10,15A 10,...,75C 16,150A 16,150B 16,150C 16,15A 16,15B 16,15C 16,225A 16,225B 16,225C 16
OTUId,,,,,,,,,,,,,,,,,,,,,
OTU_1,24014,12430,477,231,35910,9801,6183,22212,5628,239,...,5380,4219,7319,16424,187,198,407,10191,10663,3043
OTU_2,20188,24781,13429,22430,22253,24440,684,688,622,12923,...,5242,380,462,1213,5432,6266,8964,1274,660,212
OTU_3,2417,5259,29605,790,591,8685,1698,318,105,785,...,9096,1619,396,545,5873,3991,423,689,326,70
OTU_5,289,363,208,259,366,331,18787,10820,5755,176,...,90,2536,1912,74,67,83,88,2604,4620,2865
OTU_4,98,98,5834,856,361,192,72,199,72,17345,...,254,34,119,103,1671,4136,2690,47,0,9


In [11]:
#Make list of indecies and column headers
samples = list(oad_cleaned.index.values)
sites = list(oad_cleaned)

## Data of OAD Normalized (with samples as the rows and sites as the columns)

In [12]:
#Normalize cleaned data
# use this one: standard scalar - 0 value of the vector = 0, and the unit variance - will be between -0.5-0.5
# also make labels as species/samples not PC1 
scaler = preprocessing.StandardScaler()
scaler_df = scaler.fit_transform(oad_cleaned)
oad_norm = pd.DataFrame(scaler_df, index=samples, columns=sites)
#oad_norm.columns = columns
oad_norm

C:\Users\Meenakshi Krishnan\Miniconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Meenakshi Krishnan\Miniconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,45A 10,45B 10,45C 10,75A 10,75B 10,75C 10,150A 10,150B 10,150C 10,15A 10,...,75C 16,150A 16,150B 16,150C 16,15A 16,15B 16,15C 16,225A 16,225B 16,225C 16
OTU_1,12.094017,6.934438,0.120995,0.048391,13.477260,5.538181,4.816835,13.985559,5.727171,0.068125,...,7.180294,12.660791,14.678848,15.655486,0.242522,0.243085,0.601609,14.080376,13.899250,11.492981
OTU_2,10.151444,13.926096,6.364506,14.474022,8.315568,13.971257,0.442069,0.325602,0.505285,9.361409,...,6.993217,1.017407,0.820770,1.075109,10.320904,11.555838,15.159325,1.660153,0.758478,0.706077
OTU_3,1.128582,2.875075,14.162147,0.411647,0.128369,4.895288,1.248763,0.090786,-0.034011,0.468167,...,12.217798,4.775196,0.687384,0.434803,11.168295,7.314488,0.628829,0.845324,0.319708,0.165017
OTU_5,0.048134,0.103546,-0.008677,0.066586,0.043330,0.082806,14.844031,6.755758,5.859648,0.021966,...,0.009034,7.556385,3.751238,-0.016670,0.011939,0.028687,0.058905,3.512670,5.960663,10.814751
OTU_4,-0.048842,-0.046466,2.703336,0.454536,0.041440,0.002732,-0.044812,0.015264,-0.068434,12.601309,...,0.231357,-0.031984,0.127563,0.011127,3.094060,7.584816,4.485595,-0.048899,-0.108553,-0.067410
OTU_239,-0.068136,-0.057221,-0.087251,-0.068579,-0.068544,-0.066396,2.156493,1.778289,-0.027752,-0.077678,...,-0.079082,1.147823,0.260950,-0.068432,-0.101431,-0.049615,-0.046573,5.918157,4.537944,-0.036928
OTU_10,0.311646,-0.060052,0.440594,0.295327,0.183550,0.044209,-0.096523,0.093324,2.271296,0.771495,...,-0.061459,-0.077477,-0.078580,1.784429,0.327069,0.640187,-0.005743,-0.086507,-0.108553,-0.090272
OTU_8,-0.071690,-0.096847,-0.083877,-0.096522,0.234196,-0.099808,-0.086977,-0.041854,8.014745,-0.101124,...,-0.100772,-0.116905,2.950917,0.611174,-0.103352,-0.116731,-0.084001,-0.106007,-0.108553,-0.097893
OTU_7,-0.087938,-0.096847,-0.100748,-0.085475,-0.084796,-0.095775,-0.085386,-0.100240,10.526591,-0.100391,...,-0.112972,-0.135103,-0.112937,-0.087602,-0.116803,-0.126052,-0.090806,-0.114364,-0.108553,-0.101703
OTU_6,-0.090984,-0.096847,-0.104605,6.173712,-0.087819,-0.103840,-0.092546,-0.100875,-0.129978,-0.098193,...,-0.110261,-0.135103,-0.112937,-0.087602,-0.116803,-0.126052,-0.090806,-0.114364,-0.108553,-0.101703


In [14]:
#Compute pairwise covariance of columns (sites)
covar = oad_norm.cov()
covar.head()

,45A 10,45B 10,45C 10,75A 10,75B 10,75C 10,150A 10,150B 10,150C 10,15A 10,...,75C 16,150A 16,150B 16,150C 16,15A 16,15B 16,15C 16,225A 16,225B 16,225C 16
45A 10,1.003968,0.916000,0.334869,0.593965,0.991091,0.860017,0.265654,0.694457,0.305619,0.390495,...,0.691092,0.681628,0.749198,0.806103,0.488111,0.520682,0.649533,0.755295,0.708485,0.591556
45B 10,0.916000,1.003968,0.525612,0.812807,0.840432,0.990057,0.185988,0.415046,0.192679,0.533003,...,0.733728,0.473557,0.466423,0.502046,0.720011,0.742590,0.868884,0.499062,0.439193,0.370467
45C 10,0.334869,0.525612,1.003968,0.400220,0.233432,0.640538,0.091107,0.028313,0.022532,0.408798,...,0.880378,0.310178,0.075338,0.069628,0.934424,0.798141,0.475553,0.103281,0.050291,0.040134
75A 10,0.593965,0.812807,0.400220,1.003968,0.486603,0.817773,0.037615,0.028896,0.036260,0.574230,...,0.430581,0.076517,0.056665,0.072113,0.623414,0.697608,0.885573,0.105077,0.053099,0.051266
75B 10,0.991091,0.840432,0.233432,0.486603,1.003968,0.768932,0.283283,0.768092,0.341600,0.324196,...,0.630844,0.723748,0.824496,0.881935,0.369040,0.409850,0.541450,0.816523,0.777319,0.648193


In [63]:
#Initialize ledoit-wolf object
lw = LedoitWolf()

In [39]:
#LW Shrinkage of Covariance Matrix
#Does it use the covariance matrix or regular input matrix?
lwdf_norm = pd.DataFrame(lw.fit(covar).get_precision())
lwdf_norm

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,312.818178,-31.985635,7.135084,-19.610414,-39.386536,-27.370870,25.863106,-10.495216,-0.205404,-0.785641,...,-12.350263,-3.538056,-20.076553,-32.403667,-1.036565,-2.286317,-19.003865,-13.062530,-8.916026,-0.786994
1,-31.985635,310.243957,1.138248,-39.092429,-27.297198,-39.065022,1.799218,-0.887974,-1.872011,2.663910,...,-9.803843,1.680669,-2.123687,-4.639964,-12.816541,-9.704836,-35.383812,-4.343593,-2.221005,-2.099460
2,7.135084,1.138248,281.212290,18.843132,10.591523,-9.707836,2.505468,6.304851,7.859863,-2.024832,...,-55.831632,-18.889260,2.301304,1.754776,-50.633836,-32.220221,9.772170,3.986150,6.263465,6.782943
3,-19.610414,-39.092429,18.843132,244.651844,-11.824080,-36.061687,-13.633054,10.161415,16.727052,7.765767,...,13.276449,13.795120,18.840020,22.619141,-3.225044,-1.234276,-42.628870,7.675395,7.213154,0.241396
4,-39.386536,-27.297198,10.591523,-11.824080,309.007672,-21.675327,33.772625,-13.694591,-2.294838,-3.282983,...,-11.261558,-4.548962,-26.575903,-41.225145,4.238216,0.508327,-13.014864,-15.644909,-10.917614,-0.171052
5,-27.370870,-39.065022,-9.707836,-36.061687,-21.675327,312.037179,-1.515728,1.744621,-1.476611,6.278115,...,-17.033725,-0.143718,1.209653,0.356112,-19.708539,-12.312366,-32.463734,-1.956512,0.030304,-0.963805
6,25.863106,1.799218,2.505468,-13.633054,33.772625,-1.515728,176.859917,-37.219731,-5.587930,7.035865,...,20.059900,-55.078740,11.521961,57.202182,-6.008625,-2.275838,-14.862749,1.382844,-29.947884,-100.922542
7,-10.495216,-0.887974,6.304851,10.161415,-13.694591,1.744621,-37.219731,315.479959,4.402122,0.268592,...,-2.978755,-32.120579,-29.826930,-21.333852,6.723196,4.817400,6.242377,-24.095796,-30.025080,-39.262909
8,-0.205404,-1.872011,7.859863,16.727052,-2.294838,-1.476611,-5.587930,4.402122,84.578156,14.597491,...,7.300560,1.462355,-27.447626,-5.632686,2.263675,6.460555,1.292719,13.439149,8.940669,-3.662301
9,-0.785641,2.663910,-2.024832,7.765767,-3.282983,6.278115,7.035865,0.268592,14.597491,229.831806,...,16.771387,6.859713,0.778934,-1.912802,-12.970589,-62.032219,-42.908902,2.022989,2.062985,1.883503


In [62]:
#Take the inverse of the lw matrix
#This is the partial correlation matrix

parcor_norm = pd.DataFrame(np.linalg.inv(lwdf_norm))
parcor_norm

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.048566,0.029898,-0.021388,0.011918,0.049890,0.022131,0.002119,0.039256,0.006606,-0.008454,...,0.008112,0.027030,0.042319,0.049483,-0.013023,-0.011089,0.009059,0.041961,0.039634,0.028504
1,0.029898,0.056164,0.032746,0.064202,0.019648,0.056021,-0.040759,-0.033638,-0.029475,0.039241,...,0.026437,-0.028320,-0.025469,-0.014890,0.048919,0.052067,0.065017,-0.022273,-0.031085,-0.037056
2,-0.021388,0.032746,0.108876,0.060936,-0.042207,0.051908,-0.063472,-0.099381,-0.057541,0.053187,...,0.055128,-0.064116,-0.089708,-0.082717,0.106158,0.094160,0.065923,-0.084481,-0.095668,-0.092683
3,0.011918,0.064202,0.060936,0.105150,-0.008340,0.073396,-0.069898,-0.089972,-0.053157,0.078257,...,0.026314,-0.075172,-0.079567,-0.068226,0.085058,0.094147,0.105749,-0.074038,-0.086252,-0.085912
4,0.049890,0.019648,-0.042207,-0.008340,0.062304,0.008037,0.018495,0.065537,0.020445,-0.025864,...,-0.000163,0.046735,0.066478,0.071986,-0.036857,-0.034994,-0.012411,0.064725,0.065035,0.052482
5,0.022131,0.056021,0.051908,0.073396,0.008037,0.065427,-0.051191,-0.053685,-0.039276,0.048407,...,0.034542,-0.041476,-0.044257,-0.033280,0.067412,0.068322,0.075074,-0.040168,-0.050559,-0.054883
6,0.002119,-0.040759,-0.063472,-0.069898,0.018495,-0.051191,0.075690,0.084865,0.047021,-0.065518,...,-0.028008,0.068345,0.072257,0.060578,-0.076122,-0.081016,-0.076256,0.069458,0.081643,0.085315
7,0.039256,-0.033638,-0.099381,-0.089972,0.065537,-0.053685,0.084865,0.145127,0.066204,-0.091596,...,-0.028209,0.109715,0.131926,0.125735,-0.111895,-0.116151,-0.098561,0.126541,0.138549,0.128794
8,0.006606,-0.029475,-0.057541,-0.053157,0.020445,-0.039276,0.047021,0.066204,0.053117,-0.051784,...,-0.025861,0.049406,0.060903,0.053704,-0.065287,-0.067136,-0.058488,0.054770,0.063144,0.063237
9,-0.008454,0.039241,0.053187,0.078257,-0.025864,0.048407,-0.065518,-0.091596,-0.051784,0.091060,...,0.010768,-0.078294,-0.083411,-0.074935,0.072722,0.089368,0.090174,-0.079044,-0.088908,-0.086078
